In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma


In [2]:
class OneToOneITU():
    def __init__(self, n, m, size_params, random_seed, lbs=(1, 1), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 

        # generate problem
        np.random.seed(random_seed)
        self.A_ij = np.random.randint(1, size_params[0], size=[n, m])/  np.random.randint(1, size_params[0], size=[n, m])
        # values_A = np.array([1,2,4,5,8,10,16,20])
        # self.A_ij = np.random.choice(values_A[size_params[0]],size= [n,m])
        self.B_ij = np.random.randint(0, size_params[1], size=[n, m])

    def D_ij(self, U_i, V_j):
        return U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

#### Checking modules

**Dual Complementary Slackness**: $\forall i,j, \ U_i > U_{0} \implies \sum_j \mu_{ij} = 0, \ V_j > V_{0} \implies \sum_i \mu_{ij} = 0$

In [3]:
def check_dual_CS(self,U_i,V_j,mu_ij, output = False):

    D_CS_i = np.sum((U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1))
    D_CS_j = np.sum((V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1)) 
    display(Math(fr'\sum_{{i}} (U_{{i}} - U_{{lb}} ) * (\sum_{{j}} \mu_{{ij}} -1) = {D_CS_i}'))
    display(Math(fr'\sum_{{j}} (V_{{j}} - V_{{lb}} ) * (\sum_{{i}} \mu_{{ij}} -1) = {D_CS_j}'))
    
    if output is True:
        D_CS_i_bool = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        D_CS_j_bool = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return D_CS_i_bool, D_CS_j_bool
    
OneToOneITU.check_dual_CS = check_dual_CS

**Dual Feasibility**:  $\forall i,j,  \ A_{ij} U_i + G_{ij} V_j \geq B_{ij},$

In [4]:
def check_dual_feas(self,U_i,V_j, output = None):
 
    D_feas = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return D_feas, np.where(D_feas<0 )
    display(Math(fr'\min_{{ij}} \min( A_{{ij}} U_i + G_{{ij}} V_j -  B_{{ij}} , 0)  = {D_feas.min()}'))
    
   
OneToOneITU.check_dual_feas = check_dual_feas 

**Primal Complementary Slackness**:  $\forall i,j, \  \mu_{ij} * (A_{ij} U_i + G_{ij} V_j - B_{ij}) = 0 $

In [5]:
def check_primal_CS(self, U_i,V_j,mu_ij, output = None):

    P_CS_ij = self.D_ij(U_i,V_j) * mu_ij
    
    if output:
        return P_CS_ij
    else:
        display(Math(fr'\sum_{{ij}} \mu_{{ij}} * (A_{{ij}} U_i + G_{{ij}} V_j - B_{{ij}}) = {P_CS_ij.sum()}'))

OneToOneITU.check_primal_CS = check_primal_CS

**Primal Feasibility**:  $\forall i,j, \  \sum_j  \mu_{ij} \leq 1, \ \sum_i  \mu_{ij} \leq 1$

In [6]:
def check_primal_feas(self, mu_ij):

    display(Math(fr'\forall i, \ \sum_{{j}}  \mu_{{ij}} \leq 1: \ { np.all(np.sum(mu_ij,axis=1) <= np.ones(self.n))}'))
    display(Math(fr'\forall j, \ \sum_{{i}}  \mu_{{ij}} \leq 1: \ {np.all(np.sum(mu_ij,axis=0) <= np.ones(self.m)) }'))
    display(Markdown(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}"))

OneToOneITU.check_primal_feas = check_primal_feas

In [7]:
def check_IR(self, U_i,V_j):
    display(Math(fr'\min_{{i}} U_{{i}} = {np.min(U_i).round(3)} \geq {self.lb_U} = U_{{lb}}'))
    display(Math(fr'\min_{{j}} V_{{j}} = {np.min(V_j).round(3)} \geq  {self.lb_V} = V_{{lb}}'))

OneToOneITU.check_IR = check_IR

The following checks all conditions and prints the results

In [8]:
def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    header_size = 1

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Feasibility</h{header_size}>"))
    self.check_primal_feas(mu_ij)

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Generalized Complementary Slackness</h{header_size}>"))
    self.check_dual_feas(U_i, V_j )
    self.check_primal_CS(U_i, V_j , mu_ij)
    
    display(Markdown(f"{'_' * 4}\n<h{header_size}>Individual Rationality</h{header_size}>"))
    self.check_IR(U_i, V_j )
    self.check_dual_CS(U_i, V_j , mu_ij)

OneToOneITU.check_all = check_all

# Auction algorithms

### Algorithm

In [9]:
def ITU_auction(self, tol):

    V_j = np.ones(self.m) * self.lb_V
    mu_ij = np.zeros([self.n,self.m+1], dtype= bool)

    unassigned = mu_ij.sum(1) == 0
    n_unassigned = unassigned.sum()
    iter = 0

    while n_unassigned > 0:
        
        i = int(np.argmax(unassigned))
      
        U_ij = self.get_U_ij(V_j,i)[0]
        if np.all(U_ij <= self.lb_U):
            mu_ij[i,-1] = 1
        
        else:
            U_ij = np.concatenate((U_ij, [self.lb_U]))
            U_ij_sorted_id = np.argsort(U_ij)
            j_i = U_ij_sorted_id[-1]
            w_i = U_ij[U_ij_sorted_id[-2] ]

            mu_ij[:,j_i] = 0
            mu_ij[i,j_i] = 1
            V_j[j_i] += np.maximum(self.get_V_ij(i,j_i, w_i) - V_j[j_i], tol)

        unassigned = mu_ij.sum(1) == 0
        n_unassigned = unassigned.sum()
        iter += 1
   
    id_i , id_j  =  np.where(mu_ij[:,:-1] == 1)
    
    U_i = np.ones(self.n) * self.lb_U
    U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]    
   
    return mu_ij, U_i, V_j


OneToOneITU.ITU_auction = ITU_auction

In [10]:
def ITU_parallel_auction(self, tol):

    V_j = np.ones(self.m) * self.lb_V
    mu_ij = np.ones( self.n, dtype= int) * (self.m +1)

    unassigned = np.where(mu_ij == self.m + 1)[0] 
    n_unassigned = len(unassigned)
    iter = 0

    while n_unassigned > 0 and iter <500000:
        
        U_ij = self.get_U_ij( V_j, unassigned)
        perm_unmatched = np.all(U_ij <= self.lb_U, axis = 1)

        if np.any(perm_unmatched):
            mu_ij[unassigned] = self.m * perm_unmatched
        
        else:
    
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned,1))), axis = 1)
            U_ij_sorted_id = np.argsort(U_ij, axis = 1)
            j_i = U_ij_sorted_id[:,-1]
            w_i = U_ij[np.arange(n_unassigned),U_ij_sorted_id[:,-2] ]
      
            j_unique = np.unique(j_i)
            offers = w_i[:,None] * ( j_i[:,None] == j_unique[None,:])  - 1e22 *  ( j_i[:,None] != j_unique[None,:]) 
            i_j_among_unass = np.argmax(offers, axis = 0)
            i_j = unassigned[ i_j_among_unass]
            best_offer_j = np.max(offers, axis = 0)
            

            mu_ij[  np.any( mu_ij[:,None] == j_unique , axis = 1) ] = self.m +1
            mu_ij[i_j] = j_unique
            V_j[j_unique] += np.maximum(self.get_V_ij(i_j,j_unique, best_offer_j) - V_j[j_unique], tol)
       
            
        unassigned = np.where(mu_ij == self.m + 1)[0] #mu_ij.sum(1) == 0
        n_unassigned = len(unassigned)
        iter += 1

    
    matched_i =  mu_ij < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_ij[matched_i]]
   
    mu_ij = mu_ij[:,None] == np.arange(self.m +1)


    return mu_ij, U_i, V_j


OneToOneITU.ITU_parallel_auction = ITU_parallel_auction

## Examples


In [66]:
n_small = 300
m_small = 10
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (4,30) , random_seed= 778, lbs =(0,0), tol = 1e-10)

In [67]:
mu_small , U_small , V_small = example_small.ITU_auction(0.001)

In [68]:
mu_parallel , U_parallel , V_parallel = example_small.ITU_parallel_auction(.001)

In [65]:
np.min((U_small[:,None] + example_small.A_ij * V_small[None,:] - example_small.B_ij) / (1+example_small.A_ij))

0.0

In [63]:
# print(np.sum(mu_parallel * 1 - mu_small * 1))
# print(np.sum(U_parallel  - U_small ))
# print(np.sum(V_parallel  - V_small ))

In [69]:
example_small.check_all((mu_small[:,:-1] , U_small , V_small ))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 10 over 10

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>